In [1]:
!python starter.py

In [2]:
from starter import *
library_details()

OpenCV:  4.5.3
Numpy:  1.19.5


In [3]:
labels = ['arduino', 'microbit', 'esp32']
# labels = ['arduino']
number_imgs = 150
print(labels)

['arduino', 'microbit', 'esp32']


In [4]:
IMAGES_PATH = os.path.join('..', 'ssd_workspace', 'image_dataset', 'images')

In [5]:
if not os.path.exists(IMAGES_PATH):
    !mkdir {IMAGES_PATH}

In [6]:
VIDEOS_PATH = []
VIDEOS_PATH.append(os.path.join('..', 'ssd_workspace','videos', 'arduino.mp4'))
VIDEOS_PATH.append(os.path.join('..', 'ssd_workspace','videos','microbit.mp4'))
VIDEOS_PATH.append(os.path.join('..', 'ssd_workspace','videos','esp32.mp4'))
print(VIDEOS_PATH)

['..\\ssd_workspace\\videos\\arduino.mp4', '..\\ssd_workspace\\videos\\microbit.mp4', '..\\ssd_workspace\\videos\\esp32.mp4']


In [7]:
videos = {}
for key in labels:
    for value in VIDEOS_PATH:
        videos[key] = value
        VIDEOS_PATH.remove(value)
        break
print(videos)

{'arduino': '..\\ssd_workspace\\videos\\arduino.mp4', 'microbit': '..\\ssd_workspace\\videos\\microbit.mp4', 'esp32': '..\\ssd_workspace\\videos\\esp32.mp4'}


In [8]:
for label in labels:
    print("Collecting images for {}".format(label))
    try:
        cap = cv2.VideoCapture(videos[label]) 

        img_num = 0
        while(cap.isOpened()):  
            ret,frame = cap.read()

            cv2.imshow('video',frame)
            """
            if cv2.waitKey(1) & 0xFF == ord('q'):      #To check if 'q' key is pressed 
                break

            if cv2.waitKey(33) == ord('a'):
                print("pressed a")    """
            if img_num == 25:
                print("Number of images collected for {} : {}".format(label, img_num))
                break

            k = cv2.waitKey(1)
            if k & 0xFF == ord('q'):
                break

            if k & 0xFF == ord('a'):
                img_name = os.path.join(IMAGES_PATH, label+'_'+'{}.jpg'.format(img_num))
                cv2.imwrite(img_name, frame)
                img_num = img_num + 1
    #             print("image saved {}".format(img_num))

            time.sleep(0.05)
    except cv2.error as error:
        print("Video ended")
#         print("[Error]: {}".format(error))


    cap.release()
    cv2.destroyAllWindows()

Number of images collected for arduino : 25
Number of images collected for microbit : 25
Number of images collected for esp32 : 25


In [9]:
cap.release()
cv2.destroyAllWindows()

In [22]:
import glob
import os
import numpy as np
import sys

current_dir = IMAGES_PATH
split_pct = 10;
# train_set = os.path.join('..', 'ssd_workspace', 'image_dataset', 'train_test_set')
# test_set = os.path.join('..', 'ssd_workspace', 'image_dataset', 'train_test_set')
file_train = open("train_test_set/train.txt", "w")  
file_test = open("train_test_set/test.txt", "w")  
counter = 1  
index_test = round(100 / split_pct)  
for pathAndFilename in glob.iglob(os.path.join(current_dir, "*.jpg")):  
        title, ext = os.path.splitext(os.path.basename(pathAndFilename))
        if counter == index_test:
                counter = 1
                file_test.write(current_dir + "/" + title + '.jpg' + "\n")
        else:
                file_train.write(current_dir + "/" + title + '.jpg' + "\n")
                counter = counter + 1
file_train.close()
file_test.close()

In [40]:

import os
import sys
import random

import numpy as np
import tensorflow as tf

#import xml.etree.elementtree as et
import xml.etree.ElementTree as et

#There are only two definitions of my tags, depending on my own pictures
VOC_LABELS = {
    'none': (0, 'Background'),
    'arduino': (1, 'arduino'),
    'microbit': (2, 'microbit'),
    'esp32': (2, 'esp32'),
}

#A folder for pictures and labels
DIRECTORY_ANNOTATIONS = 'F:/github/ssd_workspace/image_dataset/annotations/'
DIRECTORY_IMAGES = 'F:/github/ssd_workspace/image_dataset/images/'

#Random seed
RANDOM_SEED = 4242
SAMPLES_PER_FILES = 1 #samples per file


#Generate integer, floating point, and string properties
def int64_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def float_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def bytes_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


#Image processing
def _process_image(directory, name):
    # Read the image file.
    filename = directory + DIRECTORY_IMAGES + name + '.jpg'
    image_data = tf.io.gfile.GFile(filename, 'rb').read()

    # Read the XML annotation file.
    filename = os.path.join(directory, DIRECTORY_ANNOTATIONS, name + '.xml')
    tree = et.parse(filename)
    root = tree.getroot()

    # Image shape.
    size = root.find('size')
    shape = [int(size.find('height').text),
             int(size.find('width').text),
             int(size.find('depth').text)]
    # Find annotations.
    bboxes = []
    labels = []
    labels_text = []
    difficult = []
    truncated = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        labels.append(int(VOC_LABELS[label][0]))
        labels_text.append (label.encode ('ascii')) #ාchange to ASCII format

        if obj.find('difficult'):
            difficult.append(int(obj.find('difficult').text))
        else:
            difficult.append(0)
        if obj.find('truncated'):
            truncated.append(int(obj.find('truncated').text))
        else:
            truncated.append(0)

        bbox = obj.find('bndbox')

        ####################################################################		
        bboxes.append((max(float(bbox.find('ymin').text) / shape[0],0.0),
                   max(float(bbox.find('xmin').text) / shape[1],0.0),
                   min(float(bbox.find('ymax').text) / shape[0],1.0),
                   min(float(bbox.find('xmax').text) / shape[1],1.0)
                   ))
        ######################################################################
        # the above code in ### replaced by below commented code
        # a = float(bbox.find('ymin').text) / shape[0]
        # b = float(bbox.find('xmin').text) / shape[1]
        # a1 = float(bbox.find('ymax').text) / shape[0]
        # b1 = float(bbox.find('xmax').text) / shape[1]
        # a_e = a1 - a
        # b_e = b1 - b
        # if abs(a_e) < 1 and abs(b_e) < 1:
            # bboxes.append((a, b, a1, b1))

    return image_data, shape, bboxes, labels, labels_text, difficult, truncated


#Conversion example
def _convert_to_example(image_data, labels, labels_text, bboxes, shape,
                        difficult, truncated):
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    for b in bboxes:
        assert len(b) == 4
        # pylint: disable=expression-not-assigned
        [l.append(point) for l, point in zip([ymin, xmin, ymax, xmax], b)]
        # pylint: enable=expression-not-assigned

    image_format = b'JPEG'
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(shape[0]),
        'image/width': int64_feature(shape[1]),
        'image/channels': int64_feature(shape[2]),
        'image/shape': int64_feature(shape),
        'image/object/bbox/xmin': float_feature(xmin),
        'image/object/bbox/xmax': float_feature(xmax),
        'image/object/bbox/ymin': float_feature(ymin),
        'image/object/bbox/ymax': float_feature(ymax),
        'image/object/bbox/label': int64_feature(labels),
        'image/object/bbox/label_text': bytes_feature(labels_text),
        'image/object/bbox/difficult': int64_feature(difficult),
        'image/object/bbox/truncated': int64_feature(truncated),
        'image/format': bytes_feature(image_format),
        'image/encoded': bytes_feature(image_data)}))
    return example


#Add to tfrecord
def _add_to_tfrecord(dataset_dir, name, tfrecord_writer):
    image_data, shape, bboxes, labels, labels_text, difficult, truncated = \
        _process_image(dataset_dir, name)
    example = _convert_to_example(image_data, labels, labels_text,
                                  bboxes, shape, difficult, truncated)
    tfrecord_writer.write(example.SerializeToString())


#Name is the prefix of the converted file
def _get_output_filename(output_dir, name, idx):
    return '%s/%s_%03d.tfrecord' % (output_dir, name, idx)


def run(dataset_dir, output_dir, name='voc_train', shuffling=False):
    if not tf.io.gfile.exists(dataset_dir):
        tf.io.gfile.makedirs(dataset_dir)

    path = os.path.join(dataset_dir, DIRECTORY_ANNOTATIONS)
    filenames = sorted (os.listdir (path)) #(sort)
    if shuffling:
        random.seed(RANDOM_SEED)
        random.shuffle(filenames)

    i = 0
    fidx = 0
    while i < len(filenames):
        # Open new TFRecord file.
        tf_filename = _get_output_filename(output_dir, name, fidx)
        with tf.io.TFRecordWriter(tf_filename) as tfrecord_writer:
            j = 0
            while i < len(filenames) and j < SAMPLES_PER_FILES:
                sys.stdout.write ('converting image%d /% d \ n '% (i + 1, len (filenames))) #(terminal printing, similar to print)
                sys.stdout.flush() #(buffer)

                filename = filenames[i]
                img_name = filename[:-4]
                print(img_name)
                _add_to_tfrecord(dataset_dir, img_name, tfrecord_writer)
                i += 1
                j += 1
            fidx += 1

    print('\nFinished converting the Pascal VOC dataset!')


#Original dataset path, output path and output file name
dataset_dir = ""   #"/voc2007/"
output_dir = "../ssd_workspace/tfrecords/"
name = "voc_2007_train"


def main(_):
    run(dataset_dir, output_dir, name)


if __name__ == '__main__':
    tf.compat.v1.app.run()

converting image1 / 75 \ n arduino_0
converting image2 / 75 \ n arduino_1
converting image3 / 75 \ n arduino_10
converting image4 / 75 \ n arduino_11
converting image5 / 75 \ n arduino_12
converting image6 / 75 \ n arduino_13
converting image7 / 75 \ n arduino_14
converting image8 / 75 \ n arduino_15
converting image9 / 75 \ n arduino_16
converting image10 / 75 \ n arduino_17
converting image11 / 75 \ n arduino_18
converting image12 / 75 \ n arduino_19
converting image13 / 75 \ n arduino_2
converting image14 / 75 \ n arduino_20
converting image15 / 75 \ n arduino_21
converting image16 / 75 \ n arduino_22
converting image17 / 75 \ n arduino_23
converting image18 / 75 \ n arduino_24
converting image19 / 75 \ n arduino_3
converting image20 / 75 \ n arduino_4
converting image21 / 75 \ n arduino_5
converting image22 / 75 \ n arduino_6
converting image23 / 75 \ n arduino_7
converting image24 / 75 \ n arduino_8
converting image25 / 75 \ n arduino_9
converting image26 / 75 \ n esp32_0
converti

SystemExit: 

F:\tf_object_detection\tf-gpu-venv\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
